### The GAN model in this code is based on [this repository](https://github.com/davidADSP/GDL_code/blob/master/models/GAN.py) and just simplify for understanding.

In [1]:
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Conv2D, Activation, Dropout, Reshape
from keras.layers import BatchNormalization, UpSampling2D, Conv2DTranspose
from keras.initializers import RandomNormal
from keras.optimizers import Adam, RMSprop
from keras.utils import plot_model

import os
from os import walk, getcwd

import matplotlib.pyplot as plt

import numpy as np

weight_init = RandomNormal(mean=0., stddev=0.02)
z_dim = 100
batch_size = 64        
epochs = 2000 


Using TensorFlow backend.


In [2]:
## Discriminator part

discriminator = Sequential([
    Conv2D(filters =64, 
           kernel_size = (5,5), 
           strides = 2,
           padding = 'same',
           input_shape=(28, 28, 1),
           name = "conv0"),
    Activation('relu', name = "relu0"),
    Dropout(rate = 0.4, name = "Dropout0"),

    Conv2D(filters =64, 
           kernel_size = (5,5), 
           strides = 2,
           padding = 'same',
           name = "conv1"),
    Activation('relu', name = "relu1"),
    Dropout(rate = 0.4, name = "Dropout1"),
    
    Conv2D(filters =128, 
           kernel_size = (5,5), 
           strides = 2,
           padding = 'same',
           name = "conv2"),
    Activation('relu', name = "relu2"),
    Dropout(rate = 0.4, name = "Dropout2"),

    Conv2D(filters =128, 
           kernel_size = (5,5), 
           strides = 1,
           padding = 'same',
           name = "conv3"),
    Activation('relu', name = "relu3"),
    Dropout(rate = 0.4, name = "Dropout3"),    
    
    Flatten(),
    
    Dense(1, activation='sigmoid', kernel_initializer=weight_init),
])

discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv0 (Conv2D)               (None, 14, 14, 64)        1664      
_________________________________________________________________
relu0 (Activation)           (None, 14, 14, 64)        0         
_________________________________________________________________
Dropout0 (Dropout)           (None, 14, 14, 64)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 7, 7, 64)          102464    
_________________________________________________________________
relu1 (Activation)           (None, 7, 7, 64)          0         
_________________________________________________________________
Dropout1 (Dropout)           (None, 7, 7, 64)          0         
_________________________________________________________________
conv2 (Conv2D)               (None, 4, 4, 128)        

In [3]:
## Generator part

generator = Sequential([
    Dense(np.prod((7,7,64)), 
          kernel_initializer = weight_init,
          input_shape=(z_dim,),
          name = "dense0"),
    BatchNormalization(momentum = 0.9),
    Activation('relu', name = "relu0"),
    Reshape((7,7,64), name = "reshape0"),
    UpSampling2D(name = "upsample0"),
    Conv2D(filters = 128,
          kernel_size = (5,5),
          padding = 'same',
          strides = 1,
          name = "conv0",
          kernel_initializer = weight_init),
    BatchNormalization(momentum = 0.9),    
    
    Activation('relu', name = "relu1"),
    UpSampling2D(name = "upsample1"),
    Conv2D(filters = 64,
          kernel_size = (5,5),
          padding = 'same',
          strides = 1,
          name = "conv1",
          kernel_initializer = weight_init),
    BatchNormalization(momentum = 0.9),        
    
    Activation('relu', name = "relu2"),
    Conv2DTranspose(filters = 64,
          kernel_size = (5,5),
          padding = 'same',
          strides = 1,
          name = "conv2",
          kernel_initializer = weight_init),
    BatchNormalization(momentum = 0.9),       
    
    Activation('relu', name = "relu3"),
    Conv2DTranspose(filters = 1,
          kernel_size = (5,5),
          padding = 'same',
          strides = 1,
          name = "conv3",
          kernel_initializer = weight_init),    
    
    Activation('tanh', name = 'tanh')
])


generator.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense0 (Dense)               (None, 3136)              316736    
_________________________________________________________________
batch_normalization_1 (Batch (None, 3136)              12544     
_________________________________________________________________
relu0 (Activation)           (None, 3136)              0         
_________________________________________________________________
reshape0 (Reshape)           (None, 7, 7, 64)          0         
_________________________________________________________________
upsample0 (UpSampling2D)     (None, 14, 14, 64)        0         
_________________________________________________________________
conv0 (Conv2D)               (None, 14, 14, 128)       204928    
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 128)      

In [4]:
## Model Compile

def set_trainable(m, val):
    m.trainable = val
    for l in m.layers:
        l.trainable = val

### COMPILE DISCRIMINATOR        
discriminator.compile(optimizer=RMSprop(lr=0.0008),
                     loss = 'binary_crossentropy',
                     metrics = ['accuracy'])

### COMPILE THE FULL GAN
set_trainable(discriminator, False)

model_input = Input(shape=(100,), name='model_input')
model_output = discriminator(generator(model_input))
model = Model(model_input, model_output)
model.compile(optimizer=RMSprop(lr=0.0004),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
set_trainable(discriminator, True)

In [5]:
## Save image
def sample_images(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, z_dim))
    gen_imgs = generator.predict(noise)

    gen_imgs = 0.5 * (gen_imgs + 1)
    gen_imgs = np.clip(gen_imgs, 0, 1)

    fig, axs = plt.subplots(r, c, figsize=(15,15))
    cnt = 0

    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(np.squeeze(gen_imgs[cnt, :,:,:]), cmap = 'gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("../result/camel/sample_%d.png" % epoch)
    plt.close()

In [6]:
## Training Part

def train_discriminator(x_train, batch_size):
    valid = np.ones((batch_size,1))
    fake = np.zeros((batch_size,1))

    idx = np.random.randint(0, x_train.shape[0], batch_size)
    true_imgs = x_train[idx]
    
    noise = np.random.normal(0, 1, (batch_size, z_dim))
    gen_imgs = generator.predict(noise)
    
    d_loss_real, d_acc_real =   discriminator.train_on_batch(true_imgs, valid)
    d_loss_fake, d_acc_fake =   discriminator.train_on_batch(gen_imgs, fake)
    d_loss =  0.5 * (d_loss_real + d_loss_fake)
    d_acc = 0.5 * (d_acc_real + d_acc_fake)

    return [d_loss, d_loss_real, d_loss_fake, d_acc, d_acc_real, d_acc_fake]    
    
    
def train_generator(batch_size):
    valid = np.ones((batch_size,1))
    noise = np.random.normal(0, 1, (batch_size, z_dim))
    return model.train_on_batch(noise, valid)

def train(x_train, batch_size, epochs):
    for epoch in range(0, epochs):
        d = train_discriminator(x_train, batch_size)
        g = train_generator(batch_size)
        
        print ("%d [D loss: (%.3f)(R %.3f, F %.3f)] [D acc: (%.3f)(%.3f, %.3f)] [G loss: %.3f] [G acc: %.3f]" % (epoch, d[0], d[1], d[2], d[3], d[4], d[5], g[0], g[1]))
        
        if epoch % 50 == 0:
            sample_images(epoch)
        


In [7]:
## Load Data
def load_safari(folder):
    mypath = os.path.join("../data", folder)
    txt_name_list = []
    for (dirpath, dirnames, filenames) in walk(mypath):
        for f in filenames:
            if f != '.DS_Store':
                txt_name_list.append(f)
                break

    slice_train = int(80000/len(txt_name_list))  ###Setting value to be 80000 for the final dataset
    i = 0
    seed = np.random.randint(1, 10e6)

    for txt_name in txt_name_list:
        txt_path = os.path.join(mypath,txt_name)
        x = np.load(txt_path)
        x = (x.astype('float32') - 127.5) / 127.5
        # x = x.astype('float32') / 255.0
        
        x = x.reshape(x.shape[0], 28, 28, 1)
        
        y = [i] * len(x)  
        np.random.seed(seed)
        np.random.shuffle(x)
        np.random.seed(seed)
        np.random.shuffle(y)
        x = x[:slice_train]
        y = y[:slice_train]
        if i != 0: 
            xtotal = np.concatenate((x,xtotal), axis=0)
            ytotal = np.concatenate((y,ytotal), axis=0)
        else:
            xtotal = x
            ytotal = y
        i += 1
        
    return xtotal, ytotal

(x_train, y_train) = load_safari('camel')

In [8]:
train(x_train, batch_size = batch_size, epochs= epochs)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: (0.739)(R 0.676, F 0.801)] [D acc: (0.398)(0.797, 0.000)] [G loss: 0.675] [G acc: 1.000]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: (0.744)(R 0.534, F 0.954)] [D acc: (0.500)(1.000, 0.000)] [G loss: 0.513] [G acc: 1.000]
2 [D loss: (2.129)(R 0.408, F 3.849)] [D acc: (0.500)(1.000, 0.000)] [G loss: 0.582] [G acc: 1.000]
3 [D loss: (0.637)(R 0.580, F 0.693)] [D acc: (0.500)(1.000, 0.000)] [G loss: 0.543] [G acc: 1.000]
4 [D loss: (0.622)(R 0.548, F 0.695)] [D acc: (0.500)(1.000, 0.000)] [G loss: 0.492] [G acc: 1.000]
5 [D loss: (0.606)(R 0.507, F 0.704)] [D acc: (0.500)(1.000, 0.000)] [G loss: 0.418] [G acc: 1.000]
6 [D loss: (0.575)(R 0.432, F 0.717)] [D acc: (0.500)(1.000, 0.000)] [G loss: 0.288] [G acc: 1.000]
7 [D loss: (0.567)(R 0.318, F 0.816)] [D acc: (0.500)(1.000, 0.000)] [G loss: 0.191] [G acc: 1.000]
8 [D loss: (0.650)(R 0.232, F 1.067)] [D acc: (0.500)(1.000, 0.000)] [G loss: 0.245] [G acc: 1.000]
9 [D loss: (0.628)(R 0.273, F 0.983)] [D acc: (0.500)(1.000, 0.000)] [G loss: 0.276] [G acc: 1.000]
10 [D loss: (0.717)(R 0.292, F 1.142)] [D acc: (0.500)(1.000, 0.000)] [G loss: 0.425] [G acc: 1.000]

85 [D loss: (0.684)(R 0.669, F 0.699)] [D acc: (0.633)(0.875, 0.391)] [G loss: 0.702] [G acc: 0.250]
86 [D loss: (0.694)(R 0.640, F 0.749)] [D acc: (0.539)(0.938, 0.141)] [G loss: 0.696] [G acc: 0.422]
87 [D loss: (0.700)(R 0.645, F 0.755)] [D acc: (0.500)(0.906, 0.094)] [G loss: 0.708] [G acc: 0.094]
88 [D loss: (0.685)(R 0.656, F 0.714)] [D acc: (0.500)(0.797, 0.203)] [G loss: 0.702] [G acc: 0.125]
89 [D loss: (0.694)(R 0.657, F 0.731)] [D acc: (0.461)(0.844, 0.078)] [G loss: 0.707] [G acc: 0.109]
90 [D loss: (0.682)(R 0.666, F 0.697)] [D acc: (0.656)(0.812, 0.500)] [G loss: 0.715] [G acc: 0.016]
91 [D loss: (0.698)(R 0.641, F 0.754)] [D acc: (0.484)(0.891, 0.078)] [G loss: 0.705] [G acc: 0.062]
92 [D loss: (0.682)(R 0.661, F 0.702)] [D acc: (0.609)(0.797, 0.422)] [G loss: 0.704] [G acc: 0.172]
93 [D loss: (0.689)(R 0.641, F 0.737)] [D acc: (0.516)(0.906, 0.125)] [G loss: 0.710] [G acc: 0.047]
94 [D loss: (0.681)(R 0.659, F 0.704)] [D acc: (0.602)(0.891, 0.312)] [G loss: 0.707] [G ac

...

1655 [D loss: (0.553)(R 0.578, F 0.529)] [D acc: (0.695)(0.609, 0.781)] [G loss: 1.420] [G acc: 0.125]
1656 [D loss: (0.558)(R 0.574, F 0.542)] [D acc: (0.727)(0.625, 0.828)] [G loss: 1.459] [G acc: 0.062]
1657 [D loss: (0.567)(R 0.528, F 0.606)] [D acc: (0.734)(0.734, 0.734)] [G loss: 1.306] [G acc: 0.141]
1658 [D loss: (0.536)(R 0.558, F 0.514)] [D acc: (0.695)(0.594, 0.797)] [G loss: 1.340] [G acc: 0.094]
1659 [D loss: (0.499)(R 0.515, F 0.482)] [D acc: (0.750)(0.641, 0.859)] [G loss: 1.214] [G acc: 0.156]
1660 [D loss: (0.497)(R 0.479, F 0.515)] [D acc: (0.773)(0.734, 0.812)] [G loss: 1.369] [G acc: 0.078]
1661 [D loss: (0.472)(R 0.428, F 0.516)] [D acc: (0.742)(0.719, 0.766)] [G loss: 1.440] [G acc: 0.062]
1662 [D loss: (0.461)(R 0.500, F 0.421)] [D acc: (0.781)(0.672, 0.891)] [G loss: 1.352] [G acc: 0.156]
1663 [D loss: (0.428)(R 0.383, F 0.474)] [D acc: (0.789)(0.781, 0.797)] [G loss: 1.578] [G acc: 0.156]
1664 [D loss: (0.547)(R 0.583, F 0.511)] [D acc: (0.695)(0.578, 0.812)] [

1738 [D loss: (0.606)(R 0.686, F 0.526)] [D acc: (0.695)(0.547, 0.844)] [G loss: 1.322] [G acc: 0.094]
1739 [D loss: (0.534)(R 0.532, F 0.537)] [D acc: (0.703)(0.656, 0.750)] [G loss: 1.198] [G acc: 0.141]
1740 [D loss: (0.569)(R 0.560, F 0.578)] [D acc: (0.664)(0.594, 0.734)] [G loss: 1.397] [G acc: 0.078]
1741 [D loss: (0.472)(R 0.455, F 0.490)] [D acc: (0.773)(0.688, 0.859)] [G loss: 1.461] [G acc: 0.078]
1742 [D loss: (0.553)(R 0.609, F 0.497)] [D acc: (0.719)(0.641, 0.797)] [G loss: 1.421] [G acc: 0.109]
1743 [D loss: (0.530)(R 0.529, F 0.530)] [D acc: (0.727)(0.688, 0.766)] [G loss: 1.361] [G acc: 0.078]
1744 [D loss: (0.629)(R 0.611, F 0.647)] [D acc: (0.672)(0.609, 0.734)] [G loss: 1.249] [G acc: 0.109]
1745 [D loss: (0.482)(R 0.510, F 0.454)] [D acc: (0.719)(0.625, 0.812)] [G loss: 1.305] [G acc: 0.109]
1746 [D loss: (0.521)(R 0.555, F 0.487)] [D acc: (0.727)(0.609, 0.844)] [G loss: 1.258] [G acc: 0.094]
1747 [D loss: (0.444)(R 0.433, F 0.456)] [D acc: (0.820)(0.781, 0.859)] [

1818 [D loss: (0.528)(R 0.571, F 0.485)] [D acc: (0.711)(0.625, 0.797)] [G loss: 1.256] [G acc: 0.109]
1819 [D loss: (0.488)(R 0.484, F 0.492)] [D acc: (0.719)(0.672, 0.766)] [G loss: 1.357] [G acc: 0.156]
1820 [D loss: (0.576)(R 0.498, F 0.655)] [D acc: (0.688)(0.672, 0.703)] [G loss: 1.357] [G acc: 0.031]
1821 [D loss: (0.600)(R 0.559, F 0.640)] [D acc: (0.680)(0.641, 0.719)] [G loss: 1.250] [G acc: 0.062]
1822 [D loss: (0.521)(R 0.579, F 0.463)] [D acc: (0.711)(0.578, 0.844)] [G loss: 1.264] [G acc: 0.016]
1823 [D loss: (0.534)(R 0.505, F 0.564)] [D acc: (0.734)(0.703, 0.766)] [G loss: 1.355] [G acc: 0.047]
1824 [D loss: (0.496)(R 0.533, F 0.460)] [D acc: (0.742)(0.641, 0.844)] [G loss: 1.370] [G acc: 0.047]
1825 [D loss: (0.537)(R 0.484, F 0.591)] [D acc: (0.727)(0.672, 0.781)] [G loss: 1.215] [G acc: 0.172]
1826 [D loss: (0.565)(R 0.582, F 0.548)] [D acc: (0.656)(0.609, 0.703)] [G loss: 1.282] [G acc: 0.109]
1827 [D loss: (0.609)(R 0.605, F 0.614)] [D acc: (0.719)(0.578, 0.859)] [

1900 [D loss: (0.542)(R 0.564, F 0.519)] [D acc: (0.719)(0.641, 0.797)] [G loss: 1.188] [G acc: 0.203]
1901 [D loss: (0.501)(R 0.493, F 0.510)] [D acc: (0.758)(0.688, 0.828)] [G loss: 1.282] [G acc: 0.188]
1902 [D loss: (0.603)(R 0.515, F 0.692)] [D acc: (0.656)(0.625, 0.688)] [G loss: 1.276] [G acc: 0.109]
1903 [D loss: (0.553)(R 0.502, F 0.604)] [D acc: (0.742)(0.719, 0.766)] [G loss: 1.324] [G acc: 0.125]
1904 [D loss: (0.463)(R 0.433, F 0.492)] [D acc: (0.766)(0.734, 0.797)] [G loss: 1.360] [G acc: 0.125]
1905 [D loss: (0.561)(R 0.579, F 0.543)] [D acc: (0.703)(0.625, 0.781)] [G loss: 1.352] [G acc: 0.094]
1906 [D loss: (0.551)(R 0.592, F 0.510)] [D acc: (0.742)(0.656, 0.828)] [G loss: 1.316] [G acc: 0.094]
1907 [D loss: (0.547)(R 0.596, F 0.497)] [D acc: (0.734)(0.609, 0.859)] [G loss: 1.247] [G acc: 0.156]
1908 [D loss: (0.638)(R 0.571, F 0.705)] [D acc: (0.648)(0.609, 0.688)] [G loss: 1.269] [G acc: 0.172]
1909 [D loss: (0.558)(R 0.588, F 0.527)] [D acc: (0.695)(0.609, 0.781)] [

1983 [D loss: (0.506)(R 0.480, F 0.531)] [D acc: (0.797)(0.734, 0.859)] [G loss: 1.469] [G acc: 0.094]
1984 [D loss: (0.531)(R 0.547, F 0.515)] [D acc: (0.742)(0.625, 0.859)] [G loss: 1.323] [G acc: 0.078]
1985 [D loss: (0.569)(R 0.564, F 0.574)] [D acc: (0.688)(0.656, 0.719)] [G loss: 1.450] [G acc: 0.047]
1986 [D loss: (0.557)(R 0.684, F 0.430)] [D acc: (0.727)(0.578, 0.875)] [G loss: 1.298] [G acc: 0.078]
1987 [D loss: (0.471)(R 0.449, F 0.492)] [D acc: (0.766)(0.703, 0.828)] [G loss: 1.172] [G acc: 0.094]
1988 [D loss: (0.482)(R 0.439, F 0.525)] [D acc: (0.758)(0.719, 0.797)] [G loss: 1.248] [G acc: 0.125]
1989 [D loss: (0.576)(R 0.518, F 0.633)] [D acc: (0.703)(0.656, 0.750)] [G loss: 1.443] [G acc: 0.062]
1990 [D loss: (0.517)(R 0.520, F 0.513)] [D acc: (0.727)(0.672, 0.781)] [G loss: 1.581] [G acc: 0.000]
1991 [D loss: (0.563)(R 0.640, F 0.487)] [D acc: (0.711)(0.562, 0.859)] [G loss: 1.378] [G acc: 0.062]
1992 [D loss: (0.487)(R 0.537, F 0.436)] [D acc: (0.773)(0.656, 0.891)] [

## Result Image

### Epoch 0
![Epoch 0](../result/camel/sample_0.png)

### Epoch 100
![Epoch 0](../result/camel/sample_100.png)


### Epoch 200
![Epoch 0](../result/camel/sample_200.png)

### Epoch 500
![Epoch 0](../result/camel/sample_500.png)

### Epoch 1000
![Epoch 0](../result/camel/sample_1000.png)

### Epoch 1950
![Epoch 0](../result/camel/sample_1950.png)
